In [1]:
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.opt import Connection, message
import ib.opt
import ib.ext.Contract
 
import time
import pandas as pd

import sys

if sys.version_info.major == 2:
    import Queue as queue
else:  # >= 3
    import queue

In [3]:
class IbManager(object):
    def __init__(self, timeout=20, **kwargs):
        self.q = queue.Queue()
        self.timeout = 20

        self.con = ib.opt.ibConnection(**kwargs)
        #self.con.registerAll(self.watcher)

        self.msgs = {
            ib.opt.message.error: self.errors,
            ib.opt.message.contractDetails: self.contractDetailsHandler,
            ib.opt.message.contractDetailsEnd: self.contractDetailsHandler
        }

        self.skipmsgs = tuple(self.msgs.keys())

        for msgtype, handler in self.msgs.items():
            self.con.register(handler, msgtype)

        self.con.connect()

    def watcher(self, msg):
        if isinstance(msg, ib.opt.message.error):
            if msg.errorCode > 2000:  # informative message
                print('-' * 10, msg)

        elif not isinstance(msg, self.skipmsgs):
            print('-' * 10, msg)

    def errors(self, msg):
        if msg.id is None:  # something is very wrong in the connection to tws
            self.q.put((True, -1, 'Lost Connection to TWS'))
        elif msg.errorCode < 1000:
            self.q.put((True, msg.errorCode, msg.errorMsg))

    def contractDetailsHandler(self, msg):
        if isinstance(msg, ib.opt.message.contractDetailsEnd):
            self.q.put((False, msg.reqId, msg))
        else:
            self.q.put((False, msg.reqId, msg.contractDetails))

    def get_contract_details(self, contract):
        self.con.reqContractDetails(1, contract)

        cdetails = list()
        df = pd.DataFrame()

        while True:
            try:
                err, mid, msg = self.q.get(block=True, timeout=self.timeout)
            except queue.Empty:
                err, mid, msg = True, -1, "Timeout receiving information"
                break

            if isinstance(msg, ib.opt.message.contractDetailsEnd):
                mid, msg = None, None
                break

            cdetails.append(msg)  # must be contractDetails
            
            if df.shape[0]==0:
                df = pd.DataFrame(columns=list(msg.m_summary.__dict__.keys()))
                df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)
            else:
                df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)

        return df, err, mid, msg

In [2]:
def vixFutContract():
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_exchange = "CFE"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'VX'

    return contract

def esFutContract():
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'ES'
    
    return contract

def esOptContract(exp):
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_secType = "FOP"
    contract.m_exchange = "GLOBEX"
    #contract.m_right = "C"
    #contract.m_localSymbol = spFuts[0].m_localSymbol
    contract.m_expiry = exp
 
    return contract

In [4]:
ibm = IbManager("127.0.0.1", port=7497, clientId=999)

Server Version: 76
TWS Time at connection:20181218 10:37:01 EST


In [7]:
cdetails, err, errid, errmsg = ibm.get_contract_details(vixFutContract())
cdetails

,m_conId,m_strike,m_includeExpired,m_symbol,m_secType,m_expiry,m_right,m_exchange,m_currency,m_localSymbol,m_tradingClass,m_multiplier,m_primaryExch
0,312051008,0.0,False,VIX,FUT,20181219,None,CFE,USD,VXZ8,VX,1000,None
1,314967790,0.0,False,VIX,FUT,20190116,None,CFE,USD,VXF9,VX,1000,None
2,318249532,0.0,False,VIX,FUT,20190213,None,CFE,USD,VXG9,VX,1000,None
3,323072528,0.0,False,VIX,FUT,20190319,None,CFE,USD,VXH9,VX,1000,None
4,326501438,0.0,False,VIX,FUT,20190417,None,CFE,USD,VXJ9,VX,1000,None
5,331364306,0.0,False,VIX,FUT,20190522,None,CFE,USD,VXK9,VX,1000,None
6,334361431,0.0,False,VIX,FUT,20190619,None,CFE,USD,VXM9,VX,1000,None
7,338146256,0.0,False,VIX,FUT,20190717,None,CFE,USD,VXN9,VX,1000,None
8,343395828,0.0,False,VIX,FUT,20190821,None,CFE,USD,VXQ9,VX,1000,None


In [8]:
cdetails, err, errid, errmsg = ibm.get_contract_details(esFutContract())
cdetails

,m_conId,m_strike,m_includeExpired,m_symbol,m_secType,m_expiry,m_right,m_exchange,m_currency,m_localSymbol,m_tradingClass,m_multiplier,m_primaryExch
0,289128563,0.0,False,ES,FUT,20181221,None,GLOBEX,USD,ESZ8,ES,50,None
1,299552802,0.0,False,ES,FUT,20190315,None,GLOBEX,USD,ESH9,ES,50,None
2,310629209,0.0,False,ES,FUT,20190621,None,GLOBEX,USD,ESM9,ES,50,None
3,321454763,0.0,False,ES,FUT,20190920,None,GLOBEX,USD,ESU9,ES,50,None
4,334144679,0.0,False,ES,FUT,20191220,None,GLOBEX,USD,ESZ9,ES,50,None


In [16]:
cdetails, err, errid, errmsg = ibm.get_contract_details(esOptContract(cdetails.loc[0].m_expiry))
cdetails

,m_conId,m_strike,m_includeExpired,m_symbol,m_secType,m_expiry,m_right,m_exchange,m_currency,m_localSymbol,m_tradingClass,m_multiplier,m_primaryExch
0,300287489,1810.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1810,ES,50,None
1,300287492,1820.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1820,ES,50,None
2,300287499,1825.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1825,ES,50,None
3,300287502,1830.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1830,ES,50,None
4,300287504,1840.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1840,ES,50,None
5,300287509,1850.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1850,ES,50,None
6,300287514,1860.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1860,ES,50,None
7,300287519,1870.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1870,ES,50,None
8,300287523,1875.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1875,ES,50,None
9,300287526,1880.0,False,ES,FOP,20181221,C,GLOBEX,USD,ESZ8 C1880,ES,50,None
